In [5]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.output_parser import StrOutputParser


## Simple Chain
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

model = AzureChatOpenAI(temperature=0.3,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)
output_parser = StrOutputParser()

In [3]:
chain = prompt | model | output_parser
response = chain.invoke({"topic": "toddlers"})
print(response)

Why did the toddler bring a ladder to the playground? 

Because they wanted to reach new heights in their mischief!


In [4]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why did the frog bring a ladder to the pond?\n\nBecause it wanted to reach the "ribbit" of success!']

In [7]:
for t in chain.stream({"topic": "kids"}):
    print(t)


Why
 don
't
 scientists
 trust
 atoms
?


Because
 they
 make
 up
 everything
!



In [8]:
type(t)

str

In [11]:
for s in chain.stream({"topic": "kids"}):
    print(s, end="", flush=True)

Why don't scientists trust atoms? 

Because they make up everything!

In [11]:
response = await chain.ainvoke({"topic": "infants"})
response

'Why don\'t infants ever gamble? \n\nBecause they\'re always a little too "crib" shy!'

### More Complex Chain
#### And Runnable Map to supply user-provided inputs to the prompt.

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(chunk_size=1)
)
retriever = vectorstore.as_retriever()

In [5]:
retriever.get_relevant_documents("where did harrison work?")    

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [6]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [7]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template) 

In [9]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [10]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

Functions

In [11]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
    

In [13]:
runnable = prompt | model
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content="I'm sorry, but I cannot provide real-time information as my responses are generated based on pre-existing data. I recommend checking a reliable weather website or using a weather app to get the current weather in San Francisco.")

In [14]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [15]:
model = model.bind(functions=functions)
runnable = prompt | model
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{\n  "team_name": "patriots"\n}'}})